# Named-Entity Recognition (NER)
<br>

<font size="4">**O que é?** É uma subtarefa da área de extração da informação.</font>

<font size="4">**Objetivo?** Localizar e classificar menções de entidades nomeadas, <u>em texto não estruturado</u>, em categorias predefinidas (nomes de pessoas e organizações, locais, datas, códigos, valores monetários, etc)</font>

## Exemplo

<img src="https://github.com/ufrpe-bcc-ia/material-aulas/raw/master/imagens/ner_example1.png" width="600px">

## Como funciona?

<br>

<img src="https://github.com/ufrpe-bcc-ia/material-aulas/raw/master/imagens/ner_pipeline.jpg" width="600px">

## Aplicações reais

### 1.Classificação de conteúdo para provedores de notícias
O NER pode escanear automaticamente artigos inteiros e revelar quais são as principais pessoas, organizações e lugares discutidos neles.

<img src="https://github.com/ufrpe-bcc-ia/material-aulas/raw/master/imagens/ner_example2.png" width="600px">

<br><br><br>

### 2.Algoritmos de busca eficientes
O NER pode buscar por tags específicas e retornar um conjunto de itens (artigos) relacionados de forma eficiente.

<img src="https://github.com/ufrpe-bcc-ia/material-aulas/raw/master/imagens/ner_example3.jpeg" width="600px">

<br><br><br>

### 3.Suporte ao cliente
O NER pode categorizar a reclamação e atribuí-la ao departamento relevante dentro da organização.

<img src="https://github.com/ufrpe-bcc-ia/material-aulas/raw/master/imagens/ner_example4.png" width="600px">



## Biblioteca que iremos usar

<img src="https://github.com/ufrpe-bcc-ia/material-aulas/raw/master/imagens/ner_lib.jpeg" width="400px">

In [ ]:
'''INSTALANDO A BIBLIOTECA'''

!pip install spacy

In [ ]:
'''BAIXANDO O MODELO DA LÍNGUA PORTUGUESA'''

!python -m spacy download pt

In [ ]:
'''CLASSIFICANDO CONTEÚDO ATRAVÉS DO MODELO'''

import spacy
from spacy import displacy

text = 'Donald Trump, portador do cpf 064.468.404-62, chegou aos EUA no dia 01/06/2019. Donald Trump visitou Barack Obama e George Bush. Além disso, ele esteve em diversos estados do país pilotando uma ferrari.'

my_model = spacy.load('pt')
document = my_model(text)

print('Original Sentence: %s\n' % (text))

print(document)
    
displacy.render(document, jupyter=True, style='ent')

In [ ]:
import random
from spacy.util import minibatch, compounding

def train_model(data, labels, iterations=10, model='pt'):
    """Set up the pipeline and entity recognizer, and train the new entity."""
    random.seed(0)
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
    else:
        nlp = spacy.blank(model)  # create blank Language class
        
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner)
        
    else:
        ner = nlp.get_pipe("ner")
    
    for label in labels:
        ner.add_label(label)
    
    optimizer = nlp.begin_training()
    
    move_names = list(ner.move_names)
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        sizes = compounding(1.0, 4.0, 1.001)
        
        for itn in range(iterations):
            random.shuffle(TRAIN_DATA)
            batches = minibatch(TRAIN_DATA, size=sizes)
            losses = {}
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
    return nlp

In [ ]:
'''MELHORANDO O MODELO'''

NEW_LABELS = ['CPF', 'DATE']

TRAIN_DATA = [('Em 15/07/1988 nasceu essa linda criança', {'entities': [(3, 13, 'DATE')]}), 
              ('Data de prisão: 10/01/2018', {'entities': [(16, 26, 'DATE')]}),
              ('No dia 01/02/2016 foi decretada a setença', {'entities': [(7, 17, 'DATE')]}),
              ('A data da festa foi 07/05/2019', {'entities': [(20, 30, 'DATE')]}),
              ('João, portador do cpf 123.456.789-65, foi encontrado', {'entities': [(0, 4, 'PER'), (18, 32, 'CPF')]}),
              ('O cpf 064.468.404-62 pertence a Maria', {'entities': [(6, 20, 'CPF'), (32, 37, 'PER')]})
             ]

my_model = train_model(TRAIN_DATA, NEW_LABELS)

document = my_model(text)

print('Original Sentence: %s\n' % (text))
    
displacy.render(document, jupyter=True, style='ent')